In [ ]:
#pip install tensorflow

In [ ]:
#pip install Matplotlib

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the number of classes in your dataset
num_classes = 6

2023-07-17 14:22:46.826165: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-17 14:22:47.467618: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
import shutil
import random

# Set the path to the extracted dataset folder
dataset_folder = '/home/amit/FoodImage'

# Set the split ratio for training and testing
train_ratio = 0.8
test_ratio = 0.2

# Create directories for train and test sets
train_dir = '/home/amit/FoodImage_train'
test_dir = '/home/amit/FoodImage_test'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Iterate over subfolders in the dataset folder
for class_folder in os.listdir(dataset_folder):
    class_path = os.path.join(dataset_folder, class_folder)
    if os.path.isdir(class_path):
        # Create subdirectories in train and test directories
        train_class_dir = os.path.join(train_dir, class_folder)
        test_class_dir = os.path.join(test_dir, class_folder)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(test_class_dir, exist_ok=True)

        # Get the list of images in the current class folder
        images = os.listdir(class_path)
        random.shuffle(images)

        # Calculate the number of images for train and test sets
        num_train = int(train_ratio * len(images))
        num_test = len(images) - num_train

        # Move images to train and test subdirectories
        train_images = images[:num_train]
        test_images = images[num_train:]

        for image in train_images:
            src = os.path.join(class_path, image)
            dst = os.path.join(train_class_dir, image)
            shutil.move(src, dst)

        for image in test_images:
            src = os.path.join(class_path, image)
            dst = os.path.join(test_class_dir, image)
            shutil.move(src, dst)


In [3]:
# Set the parameters for preprocessing and data augmentation
image_size = (224, 224)
batch_size = 32

In [4]:
# Use ImageDataGenerator to load and preprocess the images
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 396 images belonging to 6 classes.
Found 101 images belonging to 6 classes.


In [5]:
import ssl

# Bypass SSL certificate verification
ssl._create_default_https_context = ssl._create_unverified_context

# Build the model architecture
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])


2023-07-17 14:23:33.032546: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-17 14:23:33.093616: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Train the model
num_epochs = 10
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)

Epoch 1/10


2023-07-17 14:23:54.946729: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


13/13 [==============================] - ETA: 0s - loss: 1.7056 - accuracy: 0.3131

2023-07-17 14:24:03.677802: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


13/13 [==============================] - 11s 700ms/step - loss: 1.7056 - accuracy: 0.3131 - val_loss: 1.1592 - val_accuracy: 0.5248
Epoch 2/10
13/13 [==============================] - 8s 618ms/step - loss: 0.8466 - accuracy: 0.7551 - val_loss: 0.7470 - val_accuracy: 0.7327
Epoch 3/10
13/13 [==============================] - 8s 628ms/step - loss: 0.5113 - accuracy: 0.8965 - val_loss: 0.5718 - val_accuracy: 0.8317
Epoch 4/10
13/13 [==============================] - 8s 646ms/step - loss: 0.3532 - accuracy: 0.9318 - val_loss: 0.4845 - val_accuracy: 0.8614
Epoch 5/10
13/13 [==============================] - 8s 619ms/step - loss: 0.2709 - accuracy: 0.9545 - val_loss: 0.4337 - val_accuracy: 0.8812
Epoch 6/10
13/13 [==============================] - 9s 653ms/step - loss: 0.2191 - accuracy: 0.9646 - val_loss: 0.4054 - val_accuracy: 0.8614
Epoch 7/10
13/13 [==============================] - 8s 642ms/step - loss: 0.1820 - accuracy: 0.9924 - val_loss: 0.3756 - val_accuracy: 0.8713
Epoch 8/10
13/13

In [8]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

2023-07-17 14:25:29.267289: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


4/4 [==============================] - 2s 379ms/step - loss: 0.3371 - accuracy: 0.8812
Test Loss: 0.33706218004226685
Test Accuracy: 0.8811880946159363


I used the MobileNetV2 pre-trained model as the base model, with additional layers added on top for classification. 

The code demonstrates the training, evaluation, and testing process using the provided datasets.

A test loss of 0.337 indicates that the model's predictions are relatively close to the true labels on average. Lower values of test loss are desirable, indicating better performance.

A test accuracy of 0.881 suggests that the model is correctly classifying around 88.1% of the food images in the test dataset. Higher values of test accuracy indicate better performance.

In [ ]:
# Test the model with external images
# Load and preprocess the external images
external_images_dir = 'path/to/external/images'
external_datagen = ImageDataGenerator(rescale=1.0/255)
external_generator = external_datagen.flow_from_directory(
    external_images_dir,
    target_size=image_size,
    batch_size=1,
    class_mode=None,
    shuffle=False
)

In [ ]:
# Make predictions on the external images
predictions = model.predict(external_generator)
predicted_classes = tf.argmax(predictions, axis=1)



In [ ]:
# Print the predicted classes
class_names = train_generator.class_indices
for i, class_index in enumerate(predicted_classes):
    class_name = list(class_names.keys())[list(class_names.values()).index(class_index)]
    print(f"Image {i+1} predicted class:", class_name)